# 如何使用示例选择器

如果您有大量示例，可能需要选择哪些示例包含在提示中。[示例选择器](/docs/concepts/example_selectors/) 是负责执行此操作的类。

基本接口定义如下：

```python
class BaseExampleSelector(ABC):
    """选择要包含在提示中的示例的接口。"""

    @abstractmethod
    def select_examples(self, input_variables: Dict[str, str]) -> List[dict]:
        """根据输入选择要使用的示例。"""
        
    @abstractmethod
    def add_example(self, example: Dict[str, str]) -> Any:
        """向存储中添加新示例。"""
```

它唯一需要定义的方法是 `select_examples` 方法。此方法接收输入变量并返回示例列表。具体实现可以决定如何选择这些示例。

LangChain 提供了几种不同类型的示例选择器。有关所有这些类型的概述，请参阅[下表](#example-selector-types)。

在本指南中，我们将逐步创建一个自定义示例选择器。

## 示例

为了使用示例选择器，我们需要创建一个示例列表。这些通常应该是示例输入和输出。为了演示目的，假设我们正在选择如何将英语翻译成意大利语的示例。

In [36]:
examples = [
    {"input": "hi", "output": "ciao"},
    {"input": "bye", "output": "arrivederci"},
    {"input": "soccer", "output": "calcio"},
]

## 自定义示例选择器

让我们编写一个示例选择器，根据单词的长度选择示例。

In [ ]:
from langchain_core.example_selectors.base import BaseExampleSelector


class CustomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        # 假设输入中包含一个 'input' 键
        new_word = input_variables["input"]
        new_word_length = len(new_word)

        # 初始化变量以存储最佳匹配及其长度差
        best_match = None
        smallest_diff = float("inf")

        # 遍历每个示例
        for example in self.examples:
            # 计算与示例中第一个单词的长度差
            current_diff = abs(len(example["input"]) - new_word_length)

            # 如果当前示例更接近长度，则更新最佳匹配
            if current_diff < smallest_diff:
                smallest_diff = current_diff
                best_match = example

        return [best_match]

In [38]:
example_selector = CustomExampleSelector(examples)

In [39]:
example_selector.select_examples({"input": "okay"})

[{'input': 'bye', 'output': 'arrivederci'}]

In [40]:
example_selector.add_example({"input": "hand", "output": "mano"})

In [41]:
example_selector.select_examples({"input": "okay"})

[{'input': 'hand', 'output': 'mano'}]

## 在提示中使用

我们现在可以在提示中使用这个示例选择器

In [42]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate.from_template("Input: {input} -> Output: {output}")

In [ ]:
prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix="Input: {input} -> Output:",
    prefix="将以下单词从英语翻译成意大利语：",
    input_variables=["input"],
)

print(prompt.format(input="word"))

Translate the following words from English to Italian:

Input: hand -> Output: mano

Input: word -> Output:


## 示例选择器类型

| 名称       | 描述                                                                                 |
|------------|---------------------------------------------------------------------------------------------|
| 相似性     | 使用输入和示例之间的语义相似性来决定选择哪些示例。                                            |
| MMR        | 使用输入和示例之间的最大边际相关性来决定选择哪些示例。                                        |
| 长度       | 根据可以容纳的长度选择示例。                                                                |
| Ngram      | 使用输入和示例之间的 ngram 重叠来决定选择哪些示例。                                          |